<a href="https://colab.research.google.com/github/rounak393/autopace/blob/main/speedtrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics opencv-python

import cv2
import torch
import shutil
import time
from ultralytics import YOLO
from google.colab import files

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

# Upload video
print("Upload your traffic video file (.mp4, .avi, etc.)")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded: {video_path}")

# Load model
model = YOLO("yolov8n.pt").to(device)
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

resize_width, resize_height = 1280, 720
if frame_width > 1280 or frame_height > 720:
    print("Resizing to 720p for better speed...")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "speed_processed_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (resize_width, resize_height))

# Imaginary line y-position
line_y = resize_height // 2

# Distance between lines in meters (real world)
real_distance_m = 10  # assume 10 meters between two points

# Track vehicles and times
vehicle_times = {}  # id -> frame number when it crosses line

frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (resize_width, resize_height))
    results = model(frame, verbose=False)

    # Draw the imaginary line
    cv2.line(frame, (0, line_y), (resize_width, line_y), (0, 0, 255), 2)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            if cls == 2 and conf > 0.4:  # class 2 = car
                center_y = (y1 + y2) // 2
                car_id = f"{x1}-{y1}-{x2}-{y2}"[:15]  # pseudo ID based on position

                # Car crossing the line
                if line_y - 5 < center_y < line_y + 5:
                    if car_id not in vehicle_times:
                        vehicle_times[car_id] = frame_count

                # Speed calculation
                if car_id in vehicle_times:
                    frames_elapsed = frame_count - vehicle_times[car_id]
                    time_sec = frames_elapsed / fps
                    if time_sec > 0:
                        speed_kmh = (real_distance_m / time_sec) * 3.6
                        cv2.putText(frame, f"{speed_kmh:.1f} km/h", (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Car {conf:.2f}', (x1, y2 + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()

print(f"Processing done in {time.time() - start_time:.2f} seconds")
shutil.move(output_path, "/content/speed_processed_video.mp4")
print("Download your processed video:")
files.download("/content/speed_processed_video.mp4")






   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

Saving sample_video-2.mp4 to sample_video-2.mp4
Uploaded: sample_video-2.mp4


100%|██████████| 6.25M/6.25M [00:00<00:00, 93.8MB/s]


Resizing to 720p for better speed...
Processing done in 22.12 seconds
Download your processed video:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>